# 매우 중요!!!!!!!! 강화학습의 많은 부분에서 사용됨
# Prioritized Experience Replay
- 이는 Q-learning이 제대로 지나가지 않는 상태 s의 transition을 우선적으로 학습시키는 기법이다.
- 우선순위를 매기는 기준은 가치함수의 벨만 방정식의 절댓값의 오차다.
- 이를 구현하기 위해서는 이진트리를 사용한다. (아마 heap을 이용한 priority queue를 사용할 듯)

In [39]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [40]:
from collections import namedtuple
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

In [41]:
ENV = 'CartPole-v0'
GAMMA = 0.99 # 시간 할인율
NUM_STEPS = 200
NUM_EPISODES = 500

In [42]:
class ReplayMemory:
    def __init__(self, CAPACITY):
        self.capacity = CAPACITY
        self.memory = []
        self.index = 0
    
    def push(self, state, action, next_state, reward):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.index] = Transition(state, action, next_state, reward)
        self.index = (self.index + 1) % self.capacity
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [43]:
TD_ERROR_EPSILON = 0.0001 # 오차에 더해줄 바이어스
class TDerrorMemory:
    def __init__(self, CAPACITY):
        self.capacity = CAPACITY
        self.memory = []
        self.index = 0
    
    def push(self, td_error):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.index] = td_error
        self.index = (self.index + 1) % self.capacity
        
    def __len__(self):
        return len(self.memory)
    
    def get_prioritized_indexes(self, batch_size):
        # TD error의 합을 구함
        sum_absolute_td_error = np.sum(np.absolute(self.memory))
        sum_absolute_td_error += TD_ERROR_EPSILON * len(self.memory) # 충분히 작은 값을 더함
        
        # batch_size개 만큼 난수를 생성하고 오름차순으로 정렬
        rand_list = np.random.uniform(0, sum_absolute_td_error, batch_size)
        rand_list = np.sort(rand_list)
        
        # 위에서 만든 난수를 인덱스로 결정
        indexs = []
        idx = 0
        tmp_sum_absolute_tf_error = 0
        # 여기서 인덱스를 결정한 방법은 다음과 같다
        # memory의 처음에 있는 값부터 절댓값과 TD_ERROR_EPSILON을 함께 더하면서 난수를 생성한 리스트와 비교하여 작을 때까지 이동하면서 선택하는 방법을 사용
        # 여기서 난수로 생성헌 리스트는 오름차순으로 정렬되어 있으므로 작은 값부터 선택이 가능하다.
        for rand_num in rand_list:
            while tmp_sum_absolute_tf_error < rand_num:
                tmp_sum_absolute_tf_error += (abs(self.memory[idx]) + TD_ERROR_EPSILON)
                idx += 1
            if idx >= len(self.memory):
                idx = len(self.memory) - 1
            indexs.append(idx)
        return indexs
    
    def update_td_error(self, updated_td_errors):
        self.memory = updated_td_errors

In [44]:
# 신경망 구성
from torch import nn
from torch.nn import functional as F

class Net(nn.Module):
    def __init__(self, n_in, n_mid, n_out):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_in, n_mid)
        self.fc2 = nn.Linear(n_mid, n_mid)
        self.fc3 = nn.Linear(n_mid, n_mid)
        self.fc4_adv = nn.Linear(n_mid, n_out)
        self.fc4_v = nn.Linear(n_mid, 1)
        
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        adv = self.fc4_adv(h3)
        val = self.fc4_v(h3).expand(-1, adv.size(1)) # 이 출력은 ReLU를 거치지 않고 adv와 덧셈을 하기 위해 열의 크기를 1개 증가시킴
        
        # val + adv에서 adv의 평균을 뺀다
        # keepdim이란 차원을 고정하고 계산을 하기 위해 사용한다
        # 따라서 adv를 열방향으로 평균(같은 행의 값에 대한 평균)을 구하면 size : (minibatch, 1)이 되므로 나중 연산을 위해 shape : (minibatch, 2)로 변경
        output = val + adv - adv.mean(1, keepdim=True).expand(-1, adv.size(1)) # 여기서 평균을 구하는 것이지만 수식으로 나타낼때는 advantage함수를 행동의 가짓수로 나누면 된다.
        return output

In [45]:
import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000

class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions

        # transition을 기억하기 위한 메모리 객체 생성
        self.memory = ReplayMemory(CAPACITY)

        # 신경망 구성
        n_in, n_mid, n_out = num_states, 32, num_actions
        self.main_q_network = Net(n_in, n_mid, n_out)  # Net 클래스를 사용
        self.target_q_network = Net(n_in, n_mid, n_out)  # Net 클래스를 사용
        print(self.main_q_network)  # 신경망의 구조를 출력
        
        self.optimizer = optim.Adam(self.main_q_network.parameters(), lr = 0.0001)
        self.td_error_memory = TDerrorMemory(CAPACITY) # TD오차를 기억하기 위한 메모리 객체 생성
    
    def replay(self, episode):
        if len(self.memory) < BATCH_SIZE:
            return
        self.batch, self.state_batch, self.action_batch, self.reward_batch, self.non_final_next_states = self.make_minibatch(episode)
        self.expected_state_action_values = self.get_expected_state_action_values() # 정답 신호로 사용할 Q(s_t, a_t)를 계산
        self.update_main_q_network() # 결합 가중치 수정
        
    def decide_action(self, state, episode):
        epsilon = 0.5 * (1 / (episode + 1))
        if epsilon < np.random.uniform(0, 1):
            self.main_q_network.eval()
            with torch.no_grad():
                action = self.main_q_network(state).max(axis=1)[1].reshape(1, 1) # 신경망 출력의 최댓값에 대한 인덱스를 얻음
        else:
            action = torch.LongTensor([[random.randrange(self.num_actions)]])
        return action
    
    def make_minibatch(self,episode):
        # 이 부분은 처음부터 prioritized Experience Replay를 사용하면 학습이 불안정해질 수 있기 때문에 
        # 처음에는 Experience Replay를 사용하고 학습이 약간 되면 Prioritized Experience Replay를 사용한다.
        if episode < 30:
            transitions = self.memory.sample(BATCH_SIZE)
        else:
            indexs = self.td_error_memory.get_prioritized_indexes(BATCH_SIZE)
            transitions = [self.memory.memory[n] for n in indexs]
        
        batch = Transition(*zip(*transitions))
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
        
        return batch, state_batch, action_batch, reward_batch, non_final_next_states
    
    def get_expected_state_action_values(self):
        self.main_q_network.eval() # 신경망을 추론 모드로 변경
        self.target_q_network.eval()
        
        # Q(s_t, a_t) 계산
        self.state_action_values = self.main_q_network(self.state_batch).gather(1, self.action_batch)
        
        # a_m 계산
        non_final_mask = torch.BoolTensor(tuple(map(lambda s: s is not None, self.batch.next_state)))
        next_state_values = torch.zeros(BATCH_SIZE)
        a_m = torch.zeros(BATCH_SIZE).type(torch.LongTensor)
        
        a_m[non_final_mask] = self.main_q_network(self.non_final_next_states).detach().max(axis=1)[1]
        a_m_non_final_next_states = a_m[non_final_mask].reshape(-1, 1)
        # 행동 a_m을 취할때의 Q값을 target_q_network로 계산
        next_state_values[non_final_mask] = self.target_q_network(self.non_final_next_states).gather(1, a_m_non_final_next_states).detach().squeeze()
        
        # 최종적인 Q값 계산
        expected_state_action_values = self.reward_batch + GAMMA * next_state_values
        return expected_state_action_values
    
    def update_main_q_network(self):
        self.main_q_network.train() # 훈련 모드로 변경
        # 손실 함수 계산
        loss = F.smooth_l1_loss(self.state_action_values, self.expected_state_action_values.unsqueeze(1))
        # 결합 가중치 수정
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
    
    def update_target_q_network(self):  # DDQN에서 추가됨
        '''Target Q-Network을 Main Q-Network와 맞춤'''
        self.target_q_network.load_state_dict(self.main_q_network.state_dict())
    
    def update_td_error_memory(self):
        self.main_q_network.eval()
        self.target_q_network.eval()
        
        # 전체 transition으로 미니배치를 생성
        transitions = self.memory.memory
        batch = Transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
        
        state_action_values = self.main_q_network(state_batch).gather(1, action_batch)
        non_final_mask = torch.BoolTensor(tuple(map(lambda s:s is not None, batch.next_state)))
        
        next_state_values = torch.zeros(len(self.memory))
        a_m = torch.zeros(len(self.memory)).type(torch.LongTensor)
        
        a_m[non_final_mask] = self.main_q_network(non_final_next_states).detach().max(axis=1)[1]
        a_m_non_final_next_states = a_m[non_final_mask].reshape(-1, 1)
        
        next_state_values[non_final_mask] = self.target_q_network(non_final_next_states).gather(1, a_m_non_final_next_states).detach().squeeze()
        
        # TD_error를 계산
        td_errors = (reward_batch + GAMMA * next_state_values) - state_action_values.squeeze()
        self.td_error_memory.memory = td_errors.detach().numpy().tolist() # tensor -> numpy -> python list로 변경함

In [46]:
class Agent:
    def __init__(self, num_states, num_actions):
        self.brain = Brain(num_states, num_actions)
        
    def update_q_function(self, episode):
        self.brain.replay(episode)
        
    def get_action(self, state, episode):
        action = self.brain.decide_action(state, episode)
        return action
    
    def memorize(self, state, action, state_next, reward):
        self.brain.memory.push(state, action, state_next, reward)
        
    def update_target_q_function(self):
        self.brain.update_target_q_network() # target_q_network을 main_q_network와 맞춤
        
    def memorize_td_error(self, td_error):
        self.brain.td_error_memory.push(td_error)
        
    def update_td_error_memory(self):
        self.brain.update_td_error_memory()

In [47]:
class Environment:
    def __init__(self):
        self.env = gym.make(ENV)
        num_states = self.env.observation_space.shape[0] # 태스크의 상태 변수 수 4를 받아옴
        num_actions = self.env.action_space.n # 태스크의 행동 가짓수 2를 받아옴
        self.agent = Agent(num_states, num_actions)
        
    def run(self):
        episode_10_list = np.zeros(10)
        complete_episodes = 0
        episode_final = False # 마지막 episode 여부
        
        for episode in range(NUM_EPISODES):
            observation = self.env.reset()
            state = observation
            state = torch.from_numpy(state).type(torch.FloatTensor)
            state = torch.unsqueeze(state, 0)
            
            for step in range(NUM_STEPS):
                action = self.agent.get_action(state, episode)
                observation_next, _, done, _ = self.env.step(action.item())
                
                # 보상을 수여
                if done:
                    state_next = None
                    # 처음에는 0 * 10개가 있는데 그 다음으로 집어 넣으려면 첫번째 부터 끝까지 선택하고 새로운 값을 집어 넣음
                    # 0 * 9, new stop ==> 0 * 8, new step, newnew step처럼
                    episode_10_list = np.hstack((episode_10_list[1:], step + 1)) # step + 1 의 경우는 가장 마지막에 시행된 step의 수임
                    if step < 195:
                        reward = torch.FloatTensor([-1.0])
                        complete_episodes = 0
                    else:
                        reward = torch.FloatTensor([1.0])
                        complete_episodes += 1
                else:
                    reward = torch.FloatTensor([0.0])
                    state_next = observation_next
                    state_next = torch.from_numpy(state_next).type(torch.FloatTensor)
                    state_next = torch.unsqueeze(state_next, 0)
                
                self.agent.memorize(state, action, state_next, reward) # 메모리에 경험을 저장함
                self.agent.memorize_td_error(0)
                self.agent.update_q_function(episode) # Experience Replay로 Q함수 저장
                state = state_next # 관측 결과 update
                
                if done:
                    print('%d Episode: Finished after %d steps：최근 10 에피소드의 평균 단계 수 = %.1lf' % (episode, step + 1, episode_10_list.mean()))
                    # TD 오차 메모리의 TD 오차를 업데이트
                    self.agent.update_td_error_memory()
                    
                    if episode % 2 == 0:
                        self.agent.update_target_q_function()
                    break
            
            if episode_final is True:
                break
            if complete_episodes >= 10:
                print("10 에피소드 연속 성공")
                episode_final = True

In [49]:
cartpole_env = Environment()
cartpole_env.run()

Net(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4_adv): Linear(in_features=32, out_features=2, bias=True)
  (fc4_v): Linear(in_features=32, out_features=1, bias=True)
)
0 Episode: Finished after 15 steps：최근 10 에피소드의 평균 단계 수 = 1.5
1 Episode: Finished after 15 steps：최근 10 에피소드의 평균 단계 수 = 3.0
2 Episode: Finished after 17 steps：최근 10 에피소드의 평균 단계 수 = 4.7
3 Episode: Finished after 24 steps：최근 10 에피소드의 평균 단계 수 = 7.1
4 Episode: Finished after 18 steps：최근 10 에피소드의 평균 단계 수 = 8.9
5 Episode: Finished after 13 steps：최근 10 에피소드의 평균 단계 수 = 10.2
6 Episode: Finished after 14 steps：최근 10 에피소드의 평균 단계 수 = 11.6
7 Episode: Finished after 13 steps：최근 10 에피소드의 평균 단계 수 = 12.9
8 Episode: Finished after 11 steps：최근 10 에피소드의 평균 단계 수 = 14.0
9 Episode: Finished after 15 steps：최근 10 에피소드의 평균 단계 수 = 15.5
10 Episode: Finished after 20 steps：최근 10 에피소드의 평균 단계 수 = 16.0
11 Episode: 